# Spam SMS Filter
## Naive Bayes, Multinomial Event Model, Laplace Smoothing

We implement a multinomial event model to filter out spam SMS. This hopefully performs better than the Bernoulli.

The dataset was obtained from https://archive.ics.uci.edu/ml/datasets/sms+spam+collection. It contains:

-> 425 spam SMS messages, manually extracted from the Grumbletext Web site which is a UK forum in which cell phone users make public claims about SMS spam messages

-> 3,375 non-spam SMS messages largely originated from Singaporeans and mostly from students attending the National University of Singapore.

-> 450 non-spam SMS messages from Caroline Tag's PhD Thesis.

-> 1,002 non-spam SMS messages and spam 322 SMS messages from the SMS Spam Corpus v.0.1 Big.

^Description paraphrased from the link above.

In [1]:
%matplotlib inline
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#getWords() takes a string as input and outputs the list of words (delimited by non-characters) in lowercase.
import re
def getWords(text):
    return re.compile('\w+').findall(text.lower())

In [3]:
#getInVec() takes a list of test words and a V-length wordlist as input.
#it outputs an n_i*1 feature vector, n_i is the number of test words
#x_j=k, k is the index of the j'th word in the wordlist
def getInVec(message, wordlist):
    x=np.array( [np.where(word==wordlist)[0] for word in message] )
    return np.array(filter(None,x)) #If the word does not appear in the wordlist, it does not affect the input vector!

In [4]:
#load the trainingset
trainingset=np.genfromtxt("data/SMSs.dat", delimiter="\t", dtype=None)
trainingset=trainingset[0:5000,:]
m=len(trainingset)
print "The length of the training set is m=" + str(m)

#generate the wordlist
wordlist = []
for i in range(0,m):
    for word in getWords( trainingset[i,1] ):
        wordlist.append( word )
print "The length of the wordlist is " + str(len(wordlist))
wordlist= np.array( list( set(wordlist) ) )
V=len(wordlist)
print "The length of the set of the wordlist is V=" + str(V)

The length of the training set is m=5000
The length of the wordlist is 78761
The length of the set of the wordlist is V=8117


In [5]:
#generate the target matrix
Y=np.array( [1 if i=="spam" else 0 for i in trainingset[:,0]] ) #Y m*1 target matrix 
Y_not=np.logical_not(Y)
sum_spam=sum(Y) #total number of spam messages
P_spam=float(sum_spam)/m

In [6]:
#F reads as lowercase PHI.
#generate the feature matrix
X=[None]*m #m*n_i, n_i is the number of words in the m'th example.
for i in range(0,m):
    SMS=getWords(trainingset[i,1])
    X[i]=getInVec(SMS, wordlist)

In [7]:
n_i=np.zeros(m) #m*1 vector representing number of words of training example
for i in range(0,m):
    n_i[i] = len(X[i])

#calculate the parameter F_k_given_spam
F_k_given_spam=np.zeros(V) #V*1 vector
F_k_given_notspam=np.zeros(V)
for k in range(0,V):
    count0=1.0 #starts at 1 for Laplace Smoothing
    count1=1.0
    denom0=V #starts at V for Laplace Smoothing
    denom1=V
    for i in range(0,m):
        if (Y[i]==1):
            count1+=sum(X[i]==k)
            denom1+=n_i[i]
        elif (Y[i]==0):
            count0+=sum (X[i]==k)
            denom0+=n_i[i]
            
    F_k_given_spam[k] = count1/denom1
    F_k_given_notspam[k] = count0/denom0

np.savetxt('F_k_given_spam-SMSFilter.txt', F_k_given_spam)
np.savetxt('F_k_given_notspam-SMSFilter.txt', F_k_given_notspam)

In [8]:
#Do not run the previous cell! It takes 10 minutes.
#The parameters were calculated and saved using the previous cell.
#Just load them.
F_k_given_spam = np.loadtxt('F_k_given_spam-SMSFilter.txt')
F_k_given_notspam = np.loadtxt('F_k_given_notspam-SMSFilter.txt')

In [9]:
#load the validationset
validationset=np.genfromtxt("data/SMSs.dat", delimiter="\t", dtype=None)
validationset=validationset[5000:,:]
P_thres=0.5
false_pos=0
true_pos=0
false_neg=0
true_neg=0

for entry in validationset:
    trial=getWords(entry[1])
    x_trial=getInVec(trial,wordlist) #feature vector of the trial
    n=len(x_trial)
    
    log_P_x_given_spam=0
    log_P_x_given_notspam=0
    for j in range(0,n):
        log_P_x_given_spam = log_P_x_given_spam + np.log(F_k_given_spam[x_trial[j]])
        log_P_x_given_notspam = log_P_x_given_notspam + np.log(F_k_given_notspam[x_trial[j]])

    P_x_given_spam = np.exp(log_P_x_given_spam)
    P_x_given_notspam = np.exp(log_P_x_given_notspam)
    
    if ((P_x_given_spam*(P_spam) + P_x_given_notspam*(1-P_spam))==0):
        print "P_x is zero for message:\n" + entry[1] + "\n"
        continue
        
    P_spam_given_x = P_spam* P_x_given_spam/ (P_x_given_spam*(P_spam) + P_x_given_notspam*(1-P_spam))
#############################################################
#print a couple of messages and the false pos/negs.
    if (true_pos<2 or (P_spam_given_x>P_thres and entry[0]=="ham") or (P_spam_given_x<=P_thres and entry[0]=="spam")):
        print "MESSAGE: '" + entry[1] + "'"
        print "The probability that this message is spam is " + str(P_spam_given_x)
        
        if (P_spam_given_x>P_thres):            
            if (entry[0]=="spam"):
                true_pos+=1
                print "Truely spam."
            else:
                false_pos+=1
                print "Falsely labelled spam."

        if (P_spam_given_x<=P_thres): 
            if (entry[0]=="spam"):
                false_neg+=1
                print "Falsely labelled nonspam."
            else:
                true_neg+=1
                print "Truely nonspam."
        print"\n"
#############################################################
    else:
        if (P_spam_given_x>P_thres):            
            if (entry[0]=="spam"):
                true_pos+=1
            else:
                false_pos+=1

        if (P_spam_given_x<=P_thres): 
            if (entry[0]=="spam"):
                false_neg+=1
            else:
                true_neg+=1

MESSAGE: 'Hmph. Go head, big baller.'
The probability that this message is spam is [ 0.00230747]
Truely nonspam.


MESSAGE: 'Well its not like you actually called someone a punto. That woulda been worse.'
The probability that this message is spam is [  8.34550573e-08]
Truely nonspam.


MESSAGE: 'Nope. Since ayo travelled, he has forgotten his guy'
The probability that this message is spam is [  4.09648660e-06]
Truely nonspam.


MESSAGE: 'You still around? Looking to pick up later'
The probability that this message is spam is [  2.11069439e-06]
Truely nonspam.


MESSAGE: 'CDs 4u: Congratulations ur awarded £500 of CD gift vouchers or £125 gift guaranteed & Freeentry 2 £100 wkly draw xt MUSIC to 87066 TnCs www.ldew.com1win150ppmx3age16'
The probability that this message is spam is [ 1.]
Truely spam.


MESSAGE: 'There's someone here that has a year  &lt;'
The probability that this message is spam is [  4.39063757e-06]
Truely nonspam.


MESSAGE: 'Guess which pub im in? Im as happy as a pig

In [10]:
print "True spams: " + str(true_pos)
print "False spams: " + str(false_pos)
print "False nonspams: " + str(false_neg)
print "True nonspams: " + str(true_neg)

True spams: 73
False spams: 2
False nonspams: 1
True nonspams: 497


This is so much better than the multi-variate Bernoulli event model.

It would be good to remove stop words like "the" or "i". 

It also does not consider capital vs lower-case. 

Also important, the training set is m=5000, while the feature length is n=8117.